In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import re

import projects_parser

In [7]:
BOOK_FILE = "../booklets_text/2016.txt"

In [8]:
with open(BOOK_FILE, encoding="utf-8") as f:
    lines = f.readlines()

In [10]:
print(lines[0:20])

['1\n', 'Project Title: Crowd Control\n', 'Name: Harry Quigley\n', 'Email: harry.quigley2@mail.dcu.ie\n', 'Programme: Computer Applications\n', 'Supervisor: Suzanne Little\n', 'Crowd Control is a people-counting and attendance management system. The system consists of a raspberry\n', 'pi, camera module, cloud backend and a web application. On the pi, using computer vision algorithms, people\n', 'entering/leaving the building are detected. The amount of people that have entered/left and the time stamp is\n', 'then sent to the server. The Crowd Control web application uses this data to display how many people are in a\n', 'venue in real time and represents trends graphically over time.\n', 'Project Area: Computer Vision, Image Video Processing, RaspberryPi, Web Application, Databases\n', 'Project Technology: HTML5, Java, JavaScript, Python, REST, SpringMVC, SQL\n', '2\n', 'Project Title: Assassin\n', 'Name: Sean Kelly\n', 'Email: sean.kelly224@mail.dcu.ie\n', 'Programme: Computer Applica

In [11]:
# Get rid of project numbering
filter_pattern = re.compile(r'^Project Number', re.M)
lines_filtered = list(filter(lambda line: not filter_pattern.match(line), lines))

In [12]:
print(lines_filtered[0:30])

['1\n', 'Project Title: Crowd Control\n', 'Name: Harry Quigley\n', 'Email: harry.quigley2@mail.dcu.ie\n', 'Programme: Computer Applications\n', 'Supervisor: Suzanne Little\n', 'Crowd Control is a people-counting and attendance management system. The system consists of a raspberry\n', 'pi, camera module, cloud backend and a web application. On the pi, using computer vision algorithms, people\n', 'entering/leaving the building are detected. The amount of people that have entered/left and the time stamp is\n', 'then sent to the server. The Crowd Control web application uses this data to display how many people are in a\n', 'venue in real time and represents trends graphically over time.\n', 'Project Area: Computer Vision, Image Video Processing, RaspberryPi, Web Application, Databases\n', 'Project Technology: HTML5, Java, JavaScript, Python, REST, SpringMVC, SQL\n', '2\n', 'Project Title: Assassin\n', 'Name: Sean Kelly\n', 'Email: sean.kelly224@mail.dcu.ie\n', 'Programme: Computer Applica

In [13]:
# Separate lines into list of projects

project_title_indices = [i for i, line in enumerate(lines_filtered) if "Project Title" in line]
print(project_title_indices[0:5])

projects_strings = []
i = 1
while i < len(project_title_indices):
    projects_strings.append("".join(lines_filtered[project_title_indices[i-1]:project_title_indices[i]]))
    i += 1
projects_strings.append("".join(lines_filtered[project_title_indices[-1]:]))

[1, 14, 27, 42, 56]


In [14]:
print(projects_strings[0:3])

['Project Title: Crowd Control\nName: Harry Quigley\nEmail: harry.quigley2@mail.dcu.ie\nProgramme: Computer Applications\nSupervisor: Suzanne Little\nCrowd Control is a people-counting and attendance management system. The system consists of a raspberry\npi, camera module, cloud backend and a web application. On the pi, using computer vision algorithms, people\nentering/leaving the building are detected. The amount of people that have entered/left and the time stamp is\nthen sent to the server. The Crowd Control web application uses this data to display how many people are in a\nvenue in real time and represents trends graphically over time.\nProject Area: Computer Vision, Image Video Processing, RaspberryPi, Web Application, Databases\nProject Technology: HTML5, Java, JavaScript, Python, REST, SpringMVC, SQL\n2\n', 'Project Title: Assassin\nName: Sean Kelly\nEmail: sean.kelly224@mail.dcu.ie\nProgramme: Computer Applications\nSupervisor: Geoff Hamilton\nAssassin is an Android multiplay

In [15]:
len(projects_strings)

206

In [16]:
regex_schema = {
    'title': r'Project Title:(.*?)Name',
    'students': r'Name:(.*?)Programme',
    'programme': r'Programme:(.*)Supervisor',
    'supervisor_and_description': r'Supervisor:(.*)Project Area',
    'area': r'Project Area:(.*?)(?:Project Technology|$)',
    'technology': r'Project Technology:(.*?)$',
}

In [17]:
projects = projects_parser.parse_projects(projects_strings, regex_schema)

In [18]:
projects[0:3]

[{'title': ' Crowd Control\n',
  'students': ' Harry Quigley\nEmail: harry.quigley2@mail.dcu.ie\n',
  'programme': ' Computer Applications\n',
  'supervisor_and_description': ' Suzanne Little\nCrowd Control is a people-counting and attendance management system. The system consists of a raspberry\npi, camera module, cloud backend and a web application. On the pi, using computer vision algorithms, people\nentering/leaving the building are detected. The amount of people that have entered/left and the time stamp is\nthen sent to the server. The Crowd Control web application uses this data to display how many people are in a\nvenue in real time and represents trends graphically over time.\n',
  'area': ' Computer Vision, Image Video Processing, RaspberryPi, Web Application, Databases\n',
  'technology': ' HTML5, Java, JavaScript, Python, REST, SpringMVC, SQL\n2'},
 {'title': ' Assassin\n',
  'students': ' Sean Kelly\nEmail: sean.kelly224@mail.dcu.ie\n',
  'programme': ' Computer Application

In [19]:
projects_final = projects_parser.canonicalize_projects(projects, name_sep="Name:", email_sep="Email:")

In [20]:
projects_final[0:3]

[{'title': 'Crowd Control',
  'students': [{'name': 'Harry Quigley',
    'email': 'harry.quigley2@mail.dcu.ie'}],
  'programme': 'Computer Applications',
  'area': 'Computer Vision, Image Video Processing, RaspberryPi, Web Application, Databases',
  'technology': 'HTML5, Java, JavaScript, Python, REST, SpringMVC, SQL 2',
  'supervisor': 'Suzanne Little',
  'description': 'Crowd Control is a people-counting and attendance management system. The system consists of a raspberry pi, camera module, cloud backend and a web application. On the pi, using computer vision algorithms, people entering/leaving the building are detected. The amount of people that have entered/left and the time stamp is then sent to the server. The Crowd Control web application uses this data to display how many people are in a venue in real time and represents trends graphically over time.'},
 {'title': 'Assassin',
  'students': [{'name': 'Sean Kelly', 'email': 'sean.kelly224@mail.dcu.ie'}],
  'programme': 'Computer 

In [21]:
len(projects_final)

206

In [22]:
projects_parser.write_json("2016", projects_final)